In [37]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import cross_val_score,GridSearchCV

from sklearn import ensemble
import xgboost

import warnings
warnings.filterwarnings('ignore')

class_col='Survived'
id_col='PassengerId'

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os 
print(os.listdir("../data")) # this will list files in input directory

# Any results you write to the current directory are saved as output.

['test.csv', 'submission.csv', 'train.csv']


# Load Data

In [38]:
raw_train=pd.read_csv('../data/train.csv')
raw_test=pd.read_csv('../data/test.csv')

In [39]:
raw_train.info() # So, there are some null in Age, Cabin and Embarked

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 741082 entries, 0 to 741081
Data columns (total 9 columns):
Id                741082 non-null int64
County            672602 non-null object
Province_State    701064 non-null object
Country_Region    741082 non-null object
Population        741082 non-null int64
Weight            741082 non-null float64
Date              741082 non-null object
Target            741082 non-null object
TargetValue       741082 non-null float64
dtypes: float64(2), int64(2), object(5)
memory usage: 50.9+ MB


In [40]:
raw_train[raw_train['County'].isna() == False]

,Id,County,Province_State,Country_Region,Population,Weight,Date,Target,TargetValue
51788,67761,Autauga,Alabama,US,55869,0.091485,2020-01-23,ConfirmedCases,0.0
51789,67762,Autauga,Alabama,US,55869,0.914848,2020-01-23,Fatalities,0.0
51790,67763,Autauga,Alabama,US,55869,0.091485,2020-01-24,ConfirmedCases,0.0
51791,67764,Autauga,Alabama,US,55869,0.914848,2020-01-24,Fatalities,0.0
51792,67765,Autauga,Alabama,US,55869,0.091485,2020-01-25,ConfirmedCases,0.0
...,...,...,...,...,...,...,...,...,...
735727,962570,Weston,Wyoming,US,6927,1.130796,2020-05-06,Fatalities,0.0
735728,962571,Weston,Wyoming,US,6927,0.113080,2020-05-07,ConfirmedCases,0.0
735729,962572,Weston,Wyoming,US,6927,1.130796,2020-05-07,Fatalities,0.0
735730,962573,Weston,Wyoming,US,6927,0.113080,2020-05-08,ConfirmedCases,0.0


In [41]:
raw_test[raw_test['County'].isna() == False]

,ForecastId,County,Province_State,Country_Region,Population,Weight,Date,Target
21780,21781,Autauga,Alabama,US,55869,0.091485,2020-04-27,ConfirmedCases
21781,21782,Autauga,Alabama,US,55869,0.914848,2020-04-27,Fatalities
21782,21783,Autauga,Alabama,US,55869,0.091485,2020-04-28,ConfirmedCases
21783,21784,Autauga,Alabama,US,55869,0.914848,2020-04-28,Fatalities
21784,21785,Autauga,Alabama,US,55869,0.091485,2020-04-29,ConfirmedCases
...,...,...,...,...,...,...,...,...
309415,309416,Weston,Wyoming,US,6927,1.130796,2020-06-08,Fatalities
309416,309417,Weston,Wyoming,US,6927,0.113080,2020-06-09,ConfirmedCases
309417,309418,Weston,Wyoming,US,6927,1.130796,2020-06-09,Fatalities
309418,309419,Weston,Wyoming,US,6927,0.113080,2020-06-10,ConfirmedCases


In [42]:
raw_test.info() # nulls in Age, Cabin and embarked ( same as train data), here in Fare Also

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311670 entries, 0 to 311669
Data columns (total 8 columns):
ForecastId        311670 non-null int64
County            282870 non-null object
Province_State    294840 non-null object
Country_Region    311670 non-null object
Population        311670 non-null int64
Weight            311670 non-null float64
Date              311670 non-null object
Target            311670 non-null object
dtypes: float64(1), int64(2), object(5)
memory usage: 19.0+ MB


In [43]:
raw_test.tail()

,ForecastId,County,Province_State,Country_Region,Population,Weight,Date,Target
311665,311666,NaN,NaN,Zimbabwe,14240168,0.607106,2020-06-08,Fatalities
311666,311667,NaN,NaN,Zimbabwe,14240168,0.060711,2020-06-09,ConfirmedCases
311667,311668,NaN,NaN,Zimbabwe,14240168,0.607106,2020-06-09,Fatalities
311668,311669,NaN,NaN,Zimbabwe,14240168,0.060711,2020-06-10,ConfirmedCases
311669,311670,NaN,NaN,Zimbabwe,14240168,0.607106,2020-06-10,Fatalities


We are merging data together, so we can process data, after feature engineering we will split 

In [44]:
data=pd.concat([raw_train, raw_test], axis=0).reset_index(drop=True) # concatinating test and train removing index
data.head(2)
data.tail(2) # surviced will be NaN for test data

,Country_Region,County,Date,ForecastId,Id,Population,Province_State,Target,TargetValue,Weight
1052750,Zimbabwe,NaN,2020-06-10,311669.0,NaN,14240168,NaN,ConfirmedCases,NaN,0.060711
1052751,Zimbabwe,NaN,2020-06-10,311670.0,NaN,14240168,NaN,Fatalities,NaN,0.607106


In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1052752 entries, 0 to 1052751
Data columns (total 10 columns):
Country_Region    1052752 non-null object
County            955472 non-null object
Date              1052752 non-null object
ForecastId        311670 non-null float64
Id                741082 non-null float64
Population        1052752 non-null int64
Province_State    995904 non-null object
Target            1052752 non-null object
TargetValue       741082 non-null float64
Weight            1052752 non-null float64
dtypes: float64(4), int64(1), object(5)
memory usage: 80.3+ MB


In [46]:
dataConfirmedCases = data[data.Target=='ConfirmedCases']
dataConfirmedCases.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 526376 entries, 0 to 1052750
Data columns (total 10 columns):
Country_Region    526376 non-null object
County            477736 non-null object
Date              526376 non-null object
ForecastId        155835 non-null float64
Id                370541 non-null float64
Population        526376 non-null int64
Province_State    497952 non-null object
Target            526376 non-null object
TargetValue       370541 non-null float64
Weight            526376 non-null float64
dtypes: float64(4), int64(1), object(5)
memory usage: 44.2+ MB


In [47]:
dataFatalities = data[data.Target=='Fatalities']
dataFatalities.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 526376 entries, 1 to 1052751
Data columns (total 10 columns):
Country_Region    526376 non-null object
County            477736 non-null object
Date              526376 non-null object
ForecastId        155835 non-null float64
Id                370541 non-null float64
Population        526376 non-null int64
Province_State    497952 non-null object
Target            526376 non-null object
TargetValue       370541 non-null float64
Weight            526376 non-null float64
dtypes: float64(4), int64(1), object(5)
memory usage: 44.2+ MB


In [48]:
dataConfirmedCases.isnull().sum()  # finding null per column

Country_Region         0
County             48640
Date                   0
ForecastId        370541
Id                155835
Population             0
Province_State     28424
Target                 0
TargetValue       155835
Weight                 0
dtype: int64

# Visualization & Data Cleaning

In [49]:
dataConfirmedCases.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 526376 entries, 0 to 1052750
Data columns (total 10 columns):
Country_Region    526376 non-null object
County            477736 non-null object
Date              526376 non-null object
ForecastId        155835 non-null float64
Id                370541 non-null float64
Population        526376 non-null int64
Province_State    497952 non-null object
Target            526376 non-null object
TargetValue       370541 non-null float64
Weight            526376 non-null float64
dtypes: float64(4), int64(1), object(5)
memory usage: 44.2+ MB


In [50]:
dataConfirmedCases = dataConfirmedCases[["Id","ForecastId","Date","Country_Region","Population","Province_State","TargetValue","Weight"]]
dataConfirmedCases.head()

,Id,ForecastId,Date,Country_Region,Population,Province_State,TargetValue,Weight
0,1.0,NaN,2020-01-23,Afghanistan,27657145,NaN,0.0,0.058359
2,3.0,NaN,2020-01-24,Afghanistan,27657145,NaN,0.0,0.058359
4,5.0,NaN,2020-01-25,Afghanistan,27657145,NaN,0.0,0.058359
6,7.0,NaN,2020-01-26,Afghanistan,27657145,NaN,0.0,0.058359
8,9.0,NaN,2020-01-27,Afghanistan,27657145,NaN,0.0,0.058359


In [53]:
dataConfirmedCases.info()
assert len(dataConfirmedCases) == len(data)/2

<class 'pandas.core.frame.DataFrame'>
Int64Index: 526376 entries, 0 to 1052750
Data columns (total 8 columns):
Id                370541 non-null float64
ForecastId        155835 non-null float64
Date              526376 non-null object
Country_Region    526376 non-null object
Population        526376 non-null int64
Province_State    497952 non-null object
TargetValue       370541 non-null float64
Weight            526376 non-null float64
dtypes: float64(4), int64(1), object(3)
memory usage: 36.1+ MB


In [56]:
dataConfirmedCases[dataConfirmedCases.ForecastId.isna() == False]

,Id,ForecastId,Date,Country_Region,Population,Province_State,TargetValue,Weight
741082,NaN,1.0,2020-04-27,Afghanistan,27657145,NaN,NaN,0.058359
741084,NaN,3.0,2020-04-28,Afghanistan,27657145,NaN,NaN,0.058359
741086,NaN,5.0,2020-04-29,Afghanistan,27657145,NaN,NaN,0.058359
741088,NaN,7.0,2020-04-30,Afghanistan,27657145,NaN,NaN,0.058359
741090,NaN,9.0,2020-05-01,Afghanistan,27657145,NaN,NaN,0.058359
...,...,...,...,...,...,...,...,...
1052742,NaN,311661.0,2020-06-06,Zimbabwe,14240168,NaN,NaN,0.060711
1052744,NaN,311663.0,2020-06-07,Zimbabwe,14240168,NaN,NaN,0.060711
1052746,NaN,311665.0,2020-06-08,Zimbabwe,14240168,NaN,NaN,0.060711
1052748,NaN,311667.0,2020-06-09,Zimbabwe,14240168,NaN,NaN,0.060711


# Modeling

In [15]:
dataConfirmedCases.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 526376 entries, 0 to 1052750
Data columns (total 10 columns):
Country_Region    526376 non-null object
County            477736 non-null object
Date              526376 non-null object
ForecastId        155835 non-null float64
Id                370541 non-null float64
Population        526376 non-null int64
Province_State    497952 non-null object
Target            526376 non-null object
TargetValue       370541 non-null float64
Weight            526376 non-null float64
dtypes: float64(4), int64(1), object(5)
memory usage: 44.2+ MB


In [44]:
data.head(1)

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Title,Fsize,FamilyLabel,TicketGroup
0,22.0,X,S,7.25,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171,Mr,2,2,1


In [45]:
#choosing required columns and converting to 0/1 for strings
data=data[['Survived','Age','Cabin','Embarked','Fare','Pclass','Sex','TicketGroup','FamilyLabel','Title']]
data=pd.get_dummies(data)
data.head()

,Survived,Age,Fare,Pclass,TicketGroup,FamilyLabel,Cabin_A,Cabin_B,Cabin_C,Cabin_D,...,Embarked_Q,Embarked_S,Sex_female,Sex_male,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Officer,Title_Royalty
0,0.0,22.0,7.2500,3,1,2,0,0,0,0,...,0,1,0,1,0,0,1,0,0,0
1,1.0,38.0,71.2833,1,2,2,0,0,1,0,...,0,0,1,0,0,0,0,1,0,0
2,1.0,26.0,7.9250,3,1,1,0,0,0,0,...,0,1,1,0,0,1,0,0,0,0
3,1.0,35.0,53.1000,1,2,2,0,0,1,0,...,0,1,1,0,0,0,0,1,0,0
4,0.0,35.0,8.0500,3,1,1,0,0,0,0,...,0,1,0,1,0,0,1,0,0,0


In [46]:
data.tail()

,Survived,Age,Fare,Pclass,TicketGroup,FamilyLabel,Cabin_A,Cabin_B,Cabin_C,Cabin_D,...,Embarked_Q,Embarked_S,Sex_female,Sex_male,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Officer,Title_Royalty
1304,NaN,21.0,8.0500,3,1,1,0,0,0,0,...,0,1,0,1,0,0,1,0,0,0
1305,NaN,39.0,108.9000,1,2,1,0,0,1,0,...,0,0,1,0,0,0,0,0,0,1
1306,NaN,38.5,7.2500,3,1,1,0,0,0,0,...,0,1,0,1,0,0,1,0,0,0
1307,NaN,21.0,8.0500,3,1,1,0,0,0,0,...,0,1,0,1,0,0,1,0,0,0
1308,NaN,9.0,22.3583,3,2,2,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0


In [47]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 26 columns):
Survived         891 non-null float64
Age              1309 non-null float64
Fare             1309 non-null float64
Pclass           1309 non-null int64
TicketGroup      1309 non-null int64
FamilyLabel      1309 non-null int64
Cabin_A          1309 non-null uint8
Cabin_B          1309 non-null uint8
Cabin_C          1309 non-null uint8
Cabin_D          1309 non-null uint8
Cabin_E          1309 non-null uint8
Cabin_F          1309 non-null uint8
Cabin_G          1309 non-null uint8
Cabin_T          1309 non-null uint8
Cabin_X          1309 non-null uint8
Embarked_C       1309 non-null uint8
Embarked_Q       1309 non-null uint8
Embarked_S       1309 non-null uint8
Sex_female       1309 non-null uint8
Sex_male         1309 non-null uint8
Title_Master     1309 non-null uint8
Title_Miss       1309 non-null uint8
Title_Mr         1309 non-null uint8
Title_Mrs        1309 non-null uint8


In [52]:
train=data[:len(raw_train)] # take train data
test=data[len(raw_train):].drop(['Survived'],axis=1) # take test data
X = train.drop(['Survived'],axis=1)
y = train.Survived

In [54]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

In [61]:
model = ensemble.RandomForestClassifier(random_state = 5, 
                                      warm_start = True,
                                      n_estimators = 26, 
                                      max_depth = 6, 
                                      max_features = 'sqrt')
clf = model.fit(X_train,y_train)
clf.score(X_test,y_test)

0.84593837535014

In [65]:
predictions = model.predict(test)
submission = pd.DataFrame({id_col: raw_test[id_col],
                           class_col: predictions.astype(np.int32)})
submission.to_csv("../output/output.csv", index=False)
